## Import các thư viện

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import random

## Tạo dữ liệu mẫu

In [2]:
# Bước 1: Tạo dữ liệu giả
random.seed(42)  # Để tạo dữ liệu có thể tái tạo
customers = [f'{i:05d}' for i in range(1, 101)]  # 100 khách hàng
products = [f'{i * 1000}' for i in range(1, 21)]  # 20 sản phẩm: 1000, 2000, ..., 20000


In [3]:
data = {
    'customers': [],
    'products': [],
    'amount': [],
    'date': []
}
for _ in range(500):
    customer_id = random.choice(customers)
    product_id = random.choice(products)
    amount = random.randint(1, 5)  # Số lượng ngẫu nhiên từ 1 đến 5
    date = f"2024-09-{random.randint(10, 30):02d}"  # Ngày ngẫu nhiên trong tháng 09/2024
    
    data['customers'].append(customer_id)
    data['products'].append(product_id)
    data['amount'].append(amount)
    data['date'].append(date)

# Tạo DataFrame từ dữ liệu giả
df = pd.DataFrame(data)

In [4]:
df.head()

,customers,products,amount,date
0,00082,4000,1,2024-09-18
1,00032,8000,2,2024-09-13
2,00087,18000,1,2024-09-28
3,00055,2000,1,2024-09-12
4,00028,8000,5,2024-09-29


In [5]:
# Bước 2: Tiền xử lý dữ liệu
# Chuyển đổi ngày tháng thành kiểu dữ liệu ngày
df['date'] = pd.to_datetime(df['date'])

In [6]:
# Tạo một DataFrame để nhóm theo sản phẩm
product_data = df.groupby('products').agg({
    'amount': 'sum',
    'date': 'max'  # Lấy ngày lớn nhất cho mỗi sản phẩm
}).reset_index()

Tính toán TF-IDF: Đầu tiên, các sản phẩm được chuyển đổi thành ma trận TF-IDF (Term Frequency-Inverse Document Frequency). TF-IDF là một phương pháp phổ biến để đại diện cho văn bản dưới dạng vector, giúp đánh giá tầm quan trọng của từ trong tài liệu so với toàn bộ tập tài liệu.

In [7]:
# Tính toán ma trận TF-IDF cho các sản phẩm
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['products'])

Tính độ tương đồng Cosine: Sau đó, phương pháp tính độ tương đồng cosine được sử dụng để đo lường sự tương đồng giữa các vector sản phẩm. Độ tương đồng cosine cho biết góc giữa hai vector, với giá trị gần 1 cho thấy chúng rất tương đồng và giá trị gần 0 cho thấy chúng khác nhau.

In [8]:
# Tính toán độ tương đồng cosine
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [9]:

# Hàm gợi ý sản phẩm cho từng khách hàng
def get_recommendations(customer_id, num_recommendations=9):
    # Lấy danh sách sản phẩm đã mua của khách hàng
    purchased_products = df[df['customers'] == customer_id]['products'].unique()
    
    recommendations = set()
    
    # Lấy gợi ý dựa trên các sản phẩm đã mua
    for product_id in purchased_products:
        # Kiểm tra xem sản phẩm có trong ma trận không
        if product_id in tfidf.get_feature_names_out():  
            idx = np.where(tfidf.get_feature_names_out() == product_id)[0][0]
            
            # Lấy độ tương đồng với các sản phẩm khác
            sim_scores = list(enumerate(cosine_sim[idx]))
            
            # Sắp xếp sản phẩm theo độ tương đồng
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            
            # Thêm sản phẩm gợi ý vào tập hợp, bỏ qua sản phẩm đã mua
            for i in sim_scores[1:]:
                if i[0] < len(tfidf.get_feature_names_out()):  # Kiểm tra chỉ số hợp lệ
                    recommendations.add(tfidf.get_feature_names_out()[i[0]])
                if len(recommendations) >= num_recommendations:
                    break

    return list(recommendations)

# Kiểm tra gợi ý cho một khách hàng cụ thể
customer_id = '00032'  # Thay đổi ID khách hàng nếu cần
recommendations = get_recommendations(customer_id)
print(f"Sản phẩm gợi ý cho khách hàng {customer_id}: {recommendations}")


Sản phẩm gợi ý cho khách hàng 00032: ['8000', '13000', '10000', '1000', '14000', '11000', '15000', '16000', '3000', '12000']


In [10]:
# Kiểm tra gợi ý cho một khách hàng cụ thể
customer_id= '00082'  # Thay đổi ID khách hàng nếu cần
recommendations = get_recommendations(customer_id)
print(f"Sản phẩm gợi ý cho khách hàng {customer_id}: {recommendations}")

Sản phẩm gợi ý cho khách hàng 00082: ['13000', '10000', '1000', '14000', '11000', '15000', '16000', '12000', '17000']
